In [2]:
import wesszumino as wz
import numpy as np
from qiskit.quantum_info import SparsePauliOp, PauliList

In [3]:
def _single_pauli(n_qubits: int, qubit: int, char: str) -> SparsePauliOp:
    """
    Return a SparsePauliOp with 'char' acting on `qubit` and I elsewhere.
    Qiskit convention: rightmost character acts on qubit 0.
    """
    label = ["I"] * n_qubits
    label[n_qubits - 1 - qubit] = char
    return SparsePauliOp(PauliList(["".join(label)]), coeffs=[1.0], ignore_pauli_phase=True)

def number_op_on_site(n_total_qubits: int, site: int, n_site: int) -> SparsePauliOp:
    """
    n_i = (I - Z_f)/2, where Z_f is on the fermion qubit of site i.
    In your encoding the fermion qubit is the MSB within the site => local_qubit = n_site-1.
    """
    fermion_local = n_site - 1
    fermion_global = site * n_site + fermion_local

    I = _single_pauli(n_total_qubits, fermion_global, "I")
    Z = _single_pauli(n_total_qubits, fermion_global, "Z")
    return 0.5 * (I - Z)

def total_fermion_number(n_total_qubits: int, N: int, n_site: int) -> SparsePauliOp:
    Nf = SparsePauliOp(PauliList(["I" * n_total_qubits]), coeffs=[0.0], ignore_pauli_phase=True)
    for site in range(N):
        Nf = Nf + number_op_on_site(n_total_qubits, site, n_site)
    return Nf.simplify()

def commutator(A: SparsePauliOp, B: SparsePauliOp, atol_simplify: float = 1e-12) -> SparsePauliOp:
    """
    [A,B] = AB - BA using PauliOp composition.
    Note: this can grow in number of Pauli terms; fine for checks at modest sizes.
    """
    AB = A.compose(B)   # A @ B
    BA = B.compose(A)   # B @ A
    C = (AB - BA).simplify(atol=atol_simplify)
    return C

def commutator_norm_inf(C: SparsePauliOp) -> float:
    """Cheap diagnostic: max absolute Pauli coefficient after simplify."""
    if len(C.coeffs) == 0:
        return 0.0
    return float(np.max(np.abs(C.coeffs)))

def check_conservation(H: SparsePauliOp, N: int, cutoff: int, atol: float = 1e-10):
    """
    Build N_f and all n_i for your encoding and report commutator sizes.
    """
    # Your code uses D_site = 2*cutoff => n_site = log2(D_site)
    n_site = int(np.log2(2 * cutoff))
    n_total = N * n_site

    Nf = total_fermion_number(n_total, N, n_site)
    C_Nf = commutator(H, Nf)
    val_Nf = commutator_norm_inf(C_Nf)

    per_site = []
    for i in range(N):
        ni = number_op_on_site(n_total, i, n_site)
        Ci = commutator(H, ni)
        per_site.append(commutator_norm_inf(Ci))

    print(f"[H, N_f] max|coeff| = {val_Nf:.3e}  ->  {'OK' if val_Nf < atol else 'NOT 0'}")
    for i, v in enumerate(per_site):
        print(f"[H, n_{i}] max|coeff| = {v:.3e}  ->  {'OK' if v < atol else 'NOT 0'}")


In [14]:
cutoff=16
N=5

H, n_total_qubits = wz.build_wz_hamiltonian(
    cutoff=cutoff, N=N, a=1.0, potential="quadratic", boundary_condition="dirichlet"
)


In [15]:
check_conservation(H, N=N, cutoff=cutoff, atol=1e-10)

[H, N_f] max|coeff| = 0.000e+00  ->  OK
[H, n_0] max|coeff| = 2.500e-01  ->  NOT 0
[H, n_1] max|coeff| = 2.500e-01  ->  NOT 0
[H, n_2] max|coeff| = 2.500e-01  ->  NOT 0
[H, n_3] max|coeff| = 2.500e-01  ->  NOT 0
[H, n_4] max|coeff| = 2.500e-01  ->  NOT 0
